In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from ast import literal_eval

In [ ]:
df_mov = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\Dataset\movies_dataset.csv", parse_dates = ['release_date'])

In [ ]:
df_credits = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\Dataset\credits.csv")

In [ ]:
duplicadoscred = df_credits[df_credits['id'].duplicated()]

In [ ]:
df_credits = df_credits.drop_duplicates(subset = 'id')



join



In [ ]:
#convierto columna id al mismo tipo que la del otro df
df_credits['id'] = df_credits['id'].astype(str)

In [ ]:
#uso outer para que traiga todo lo de movies, mas lo que tiene en comun con credits
df_movies=df_mov.merge(df_credits, how = 'outer', on = 'id')




TRANSFORMACIONES



RELLENAR VALORES FALTANTES EN BUDGET Y REVENUE
Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [ ]:
#chequeo si hay valores nulos
print(df_movies['budget'].isnull().sum())
print(df_movies['revenue'].isnull().sum())

In [ ]:
# rellenar con 0 los nulos
df_movies['budget'] = df_movies['budget'].fillna(0)
df_movies['revenue'] = df_movies['revenue'].fillna(0)

In [ ]:
#chequeo nulos en Cast y Crew porque pueden interferir al aplanar los diccionarios
print(df_movies['cast'].isnull().sum())
print(df_movies['crew'].isnull().sum())

In [ ]:
# elimino nan de cast y crew
df_movies = df_movies.dropna(subset=['cast'])
df_movies = df_movies.dropna(subset=['crew'])

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

COLUMNA RELEASE YEAR

In [ ]:
# separo la release date en 3, expando las columnas (anio, mes, dia)
df_movies[['release_year','release_month', 'release_day']]=df_movies.release_date.str.split(pat='-',expand=True)

In [ ]:
#elimino columnas que sobran
del df_movies['release_month']
del df_movies['release_day']

cambio formato de release date

In [ ]:
# pasar a formato fecha. los datos que no son strings los convierte en nulos
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format="%Y/%m/%d", errors = 'coerce')

elimino nulos de release date

In [ ]:
print(df_movies['release_date'].isnull().sum())

In [ ]:
df_movies = df_movies.dropna(subset=['release_date'])

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [ ]:
def calcularRetorno(row):
    row['revenue'] = pd.to_numeric(row['revenue'], errors = 'coerce')
    row['budget'] = pd.to_numeric(row['budget'], errors = 'coerce')
    if row['revenue'] == 0 or row['budget'] == 0:
        return 0
    else:
        return round(row['revenue'] / row['budget'],2)

df_movies['return'] = np.where((df_movies['revenue'] == 0) | (df_movies['budget'] == 0), 0, df_movies.apply(calcularRetorno, axis=1))

eliminar columnas que no serán utilizadas

In [ ]:
del df_movies['video']
del df_movies['imdb_id']
del df_movies['adult']
del df_movies['original_title']
del df_movies['poster_path']
del df_movies['homepage']

unnesting

In [ ]:
def desanidar(obj):
    lista = []
    for i in literal_eval (obj):
        try:
            lista.append(i['name'])
        except (ValueError, TypeError, KeyError):
            pass
    return lista

In [ ]:
df_movies['genres'] = df_movies['genres'].apply(desanidar)
df_movies['production_companies'] = df_movies['production_companies'].apply(desanidar)
df_movies['production_countries'] = df_movies['production_countries'].apply(desanidar)
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(desanidar)
df_movies['cast'] = df_movies['cast'].apply(desanidar)

In [ ]:
import ast
def desanidar2(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']
    

In [ ]:
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(desanidar2)

In [ ]:
def desanidar3(obj):
    lista = []
    for i in literal_eval (obj):
        if i['job'] == 'Director':
            try:
                lista.append(i['name'])
            except (ValueError, TypeError, KeyError):
                pass
    return lista

In [ ]:
df_movies['crew'] = df_movies['crew'].apply(desanidar3)




EXPORTO ARCHIVOS INDIVIDUALES QUE VOY A USAR PARA LA API




In [ ]:
#df fecha estreno
df_estreno = df_movies[['title', 'release_date']]
df_estreno.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_estreno.csv")


In [ ]:
#df puntuacion
df_puntuacion = df_movies[['title', 'release_year', 'popularity']]
df_puntuacion.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_popularidad.csv")


In [ ]:
#df votos
df_votos = df_movies[['title', 'release_year', 'vote_count', 'vote_average']]
df_votos.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_votos.csv")


In [ ]:
#df directores
df_directores = df_movies[['title', 'crew', 'return','release_year','budget', 'revenue']]
df_directores.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_director.csv")


In [ ]:
#df actores
df_actores = df_movies[['title', 'cast', 'return']]
df_actores.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\peliculas_por_actor.csv")


In [ ]:
#df modelo
df_movies = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv")
campos_mod = ['title', 'genres', 'id', 'release_year', 'runtime', 'popularity', 'cast', 'crew']
df_modelo = df_movies[campos_mod]
columnas = ['popularity', 'release_year']  # Nombres de las columnas por las cuales deseas ordenar
df_modelo = df_modelo.sort_values(by=columnas, ascending=False)
df_modelo = df_modelo[(df_modelo['runtime'] == 0.0) | (df_modelo['runtime']>40)]
df_modelo['genres'] = df_modelo['genres'].str.replace("[\[\]',]", "", regex=True)
df_modelo['cast'] = df_modelo['cast'].str.replace("[\[\]',]", "", regex=True)
df_modelo['crew'] = df_modelo['crew'].str.replace("[\[\]',]", "", regex=True)
df_modelo = df_modelo.head(6000)
df_modelo.reset_index(drop=True, inplace=True)
df_modelo.reset_index(inplace=True)
df_modelo


In [ ]:
df_modelo.to_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\modelo.csv")


In [ ]:
# creo y exporto crew
crew = df_movies[['crew', 'id']]
crew.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\crew_final.csv')


In [ ]:
#exporto csv completo limpio
df_movies.to_csv(r'C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv')
